In [1]:
"""
Lagrangian DA for the 2-layer QG system
"""

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.gridspec import GridSpec
from scipy.stats import gaussian_kde
from Lagrange_tracer import Lagrange_tracer_model
import h5py
from scipy import sparse

# fix the random seed
np.random.seed(2024)


# load data

In [ ]:
# load data
data_path = '/Users/ree/Documents/Research/Proj_1_LangrangeDA/code/qg_2layer_topo/QG_DATA_topo10_nu5e-14_K128_0312.mat'
with h5py.File(data_path, 'r') as file:
#     print("Keys: %s" % file.keys())
    psi1_hat_t = np.transpose(file['psi_1_t'][()], axes=(2, 1, 0)) # reorder the dimensions from Python's row-major order back to MATLAB's column-major order 
    dt = file['dt'][()][0,0]
    params_dataset = file['params']
    topo = params_dataset['H'][()] [0,0]
    kd = params_dataset['kd'][()] [0,0]
    U = params_dataset['U'][()] [0,0]
    kb = params_dataset['kb'][()] [0,0]
    beta = kb**2
    K = int(params_dataset['N'][()] [0,0])
    H = params_dataset['H'][()] [0,0]
    topo = file['topo'][()]
print('psi_hat_t.shape',psi1_hat_t.shape)
psi1_hat_t = psi1_hat_t['real'] + 1j * psi1_hat_t['imag']
print('psi_hat_t.dtype',psi1_hat_t.dtype)

psi_hat_t.shape (128, 128, 10000)


# generate tracer observations

In [3]:
L = 4 # number of tracers
K = psi1_hat_t.shape[0]
N = psi1_hat_t.shape[2]
kx = np.fft.fftfreq(K) * K
ky = np.fft.fftfreq(K) * K
sigma_xy = 0.1
x0 = np.random.uniform(-np.pi, np.pi, L)
y0 = np.random.uniform(-np.pi, np.pi, L)

# run model
model = Lagrange_tracer_model(N, L, kx, ky, psi1_hat_t, dt, sigma_xy, x0, y0, interv=1)
xt, yt, ut, vt = model.forward()   

/Users/ree/Documents/Research/Proj_1_LangrangeDA/code/Lagrange_tracer.py:99: ComplexWarning: Casting complex values to real discards the imaginary part
  self.ut[:,:,l] = u_ifft_shift[::self.interv, ::self.interv] # only save the sparsely sampled grids
/Users/ree/Documents/Research/Proj_1_LangrangeDA/code/Lagrange_tracer.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  self.vt[:,:,l] = v_ifft_shift[::self.interv, ::self.interv] # only save the sparsely sampled grids


In [5]:
# save data
obs = {
    'xt': xt,
    'yt': yt,
    'ut': ut,
    'vt': vt,
    'sigma_xy': sigma_xy,
    'L': L,
    'dt': dt
}
np.savez('../data/obs_K128.npz', **obs)

# Lagrangian DA

In [2]:
# load data
obs = np.load('../data/obs_K128.npz')
xt = obs['xt']
yt = obs['yt']
sigma_xy = obs['sigma_xy']
dt = obs['dt']
params = np.load('../data/est_paras_ou_K128.npy', allow_pickle=True).item()
gamma, omega, f, sigma = params['gamma'], params['omega'], params['f'], params['sigma']
eigens = np.load('../data/eigens_K128.npy', allow_pickle=True)
omega1 = eigens.item()['omega1']
omega2 = eigens.item()['omega2']
r1 = eigens.item()['r1']
r2 = eigens.item()['r2']
file = np.load('../data/psi_k_truth_K128.npz') # obtain the true underlying flow ($\boldsymbol{\Psi}^\bot$)
psi_t_k = file['psi_t_k']
tau_t_k = file['tau_t_k']
cut = file['cut']

In [26]:
cut = 1
index_to_remove = np.zeros((K, K), dtype=bool)
index_to_remove[(K//2-cut):(K//2+cut+1), :] = True
index_to_remove[:, (K//2-cut):(K//2+cut+1)] = True
~index_to_remove

array([[ True,  True,  True, False, False, False,  True,  True],
       [ True,  True,  True, False, False, False,  True,  True],
       [ True,  True,  True, False, False, False,  True,  True],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [ True,  True,  True, False, False, False,  True,  True],
       [ True,  True,  True, False, False, False,  True,  True]])

In [30]:
import numpy as np

def truncate(kk, cut):
    K = kk.shape[0]

    if kk.ndim == 1:
        index_to_remove = np.zeros(K, dtype=bool)
        index_to_remove[(K//2-cut):(K//2+cut+1)] = True
    elif kk.ndim == 2:
        index_to_remove = np.zeros((K, K), dtype=bool)
        index_to_remove[(K//2-cut):(K//2+cut+1), :] = True
        index_to_remove[:, (K//2-cut):(K//2+cut+1)] = True

    return kk[~index_to_remove]

K = 8; cut = 2
kx = np.fft.fftfreq(K) * K
truncate(kx, cut)

array([ 0.,  1., -1.])

In [12]:
# Sample array
arr = np.array([1, 2, 3, 4, 5])

# Index of elements to remove, let's say we want to remove the element at index 1 (which is 2)
# For multiple indices, use a list: [1, 3]
index_to_remove = 1

# Remove the element
new_arr = np.delete(arr, index_to_remove)

print(new_arr)

[1 3 4 5]


In [ ]:
# Lagrangian DA
def truncate(kk, cut):
    K = kk.shape[0]

    if kk.ndim == 1:
        index_to_remove = np.zeros(K, dtype=bool)
        index_to_remove[(K//2-cut):(K//2+cut+1)] = True
    elif kk.ndim == 2:
        index_to_remove = np.zeros((K, K), dtype=bool)
        index_to_remove[(K//2-cut):(K//2+cut+1), :] = True
        index_to_remove[:, (K//2-cut):(K//2+cut+1)] = True

    return kk[~index_to_remove]

def get_A(x,y,K,r1,r2,cut):
    L = x.shape[0]
    kx = np.fft.fftfreq(K) * K
    ky = np.fft.fftfreq(K) * K
    KX, KY = np.meshgrid(kx, ky)
    KX = truncate(KX, cut)
    KY = truncate(KY, cut)
    r1 = truncate(r1[:,:,0], cut)
    r2 = truncate(r2[:,:,0], cut)
    E = np.zeros((2*L, (K-2*cut+1)**2), dtype=np.complex_)
#     R_bot = np.zeros((K**2, 2*K**2), dtype=np.float_)

    exp_term = np.exp(1j * x[:,None] @ KX.flatten()[None,:] + 1j * y[:,None] @ KY.flatten()[None,:])
    E[:L,:] = exp_term * (1j) * KY.flatten()
    E[L:,:] = exp_term * (-1j) * KX.flatten()

    R_bot = sparse.block_diag([sparse.diags(r1.flatten()), sparse.diags(r2.flatten())], format="csr")
    
    A = sparse.csr_matrix(E).dot(R_bot)
#     A = E @ R_bot

    return A

N = 10
# Sigma1 = np.array(sigma_xy)[None, None]
# invBoB = np.linalg.inv(Sigma1 @ Sigma1.T)
invBoB = 1 / sigma_xy**2
mu0 = np.concatenate((psi_t_k[:,:,0].flatten(), tau_t_k[:,:,0].flatten())) # assume the initial condition is truth
n = mu0.shape[0]
R0 = np.zeros((n, n))
mu_t = np.zeros((n, N))  # posterior mean
mu_t[:, 0] = mu0
R_t = np.zeros((n, N))  # posterior covariance
R_t[:, 0] = np.diag(R0)  # only save the diagonal elements
a0 = f.flatten()
a1 = -np.diag(gamma.flatten()) + 1j * np.diag(omega.flatten())
Sigma_u = np.diag(sigma.flatten())

for i in range(1, N):
    x0 = x[:, i - 1]
    y0 = y[:, i - 1]
    x1 = x[:, i]
    y1 = y[:, i]
    x_diff = x1 - x0
    y_diff = y1 - y0

    # Need to take into account the periodic boundary conditions
    x_diff[x_diff > np.pi] = x_diff[x_diff > np.pi] - 2 * np.pi
    x_diff[x_diff < -np.pi] = x_diff[x_diff < -np.pi] + 2 * np.pi
    y_diff[y_diff > np.pi] = y_diff[y_diff > np.pi] - 2 * np.pi
    y_diff[y_diff < -np.pi] = y_diff[y_diff < -np.pi] + 2 * np.pi

    # Matrix for filtering
    A1 = get_A(x0, y0, K, r1, r2)
    
    # Update the posterior mean and posterior covariance
    mu = mu0 + (a0 + a1 @ mu0[:,None]) * dt + (R0 @ A1.T) * InvBoB @ (np.hstack((x_diff, y_diff))[:,None] - A1 @ mu0[:,None] * dt)
    R = R0 + (a1 @ R0 + R0 @ a1.T + Sigma_u @ Sigma_u.T - (R0 @ A1.T) * InvBoB @ (R0 @ A1.T).T) * dt
    mu_t[:, i] = mu
    R_t[:, i] = np.diag(R)
    mu0 = mu
    R0 = R

# # Plot
# sel0 = 10000; sel1 = 20000 # plot time range
# interv = 10 # plot interval
# xaxis = np.arange(sel0*dt, sel1*dt, interv*dt)

# fig = plt.figure()
# widths = [8, 2]
# heights = [2, 2, 2]
# spec = fig.add_gridspec(ncols=2, nrows=3, width_ratios=widths, height_ratios=heights)

# plt.subplots_adjust(wspace=0.2, hspace=0.5)     # Adjust the overall spacing of the figure
# ax1 = fig.add_subplot(spec[0, 0])
# ax2 = fig.add_subplot(spec[1, 0])
# ax3 = fig.add_subplot(spec[2, 0])
# ax4 = fig.add_subplot(spec[0, 1])
# ax5 = fig.add_subplot(spec[1, 1])
# ax6 = fig.add_subplot(spec[2, 1])

# # plot time series
# ax1.plot(xaxis, truth[0,0,sel0:sel1:interv])
# ax1.set_xlim(sel0*dt, sel1*dt)
# ax1.set_ylabel('x')
# ax1.set_title('time series')

# ax2.plot(xaxis, truth[1,0,sel0:sel1:interv], label='truth')
# ax2.plot(xaxis, u2_means[0,sel0:sel1:interv], label='filter')
# ax2.set_xlim(sel0*dt, sel1*dt)
# ax2.set_ylabel('y')
# ax2.legend()

# ax3.plot(xaxis, truth[2,0,sel0:sel1:interv], label='truth')
# ax3.plot(xaxis, u2_means[1,sel0:sel1:interv], label='filter')
# ax3.set_xlim(sel0*dt, sel1*dt)
# ax3.set_ylabel('z')
# ax3.set_xlabel('t')

# # plot pdf
# samples = truth[0, 0, :]
# kde = gaussian_kde(samples)
# xticks = np.linspace(samples.min(), samples.max(), 100)
# p = kde.evaluate(xticks)
# ax4.plot(xticks, p)
# ax4.set_title('PDF')

# samples = truth[1, 0, :]
# kde = gaussian_kde(samples)
# xticks = np.linspace(samples.min(), samples.max(), 100)
# p = kde.evaluate(xticks)
# ax5.plot(xticks, p, label='truth')
# samples = u2_means[0, :]
# kde = gaussian_kde(samples)
# p = kde.evaluate(xticks)
# ax5.plot(xticks, p, label='filter')

# samples = truth[2, 0, :]
# kde = gaussian_kde(samples)
# xticks = np.linspace(samples.min(), samples.max(), 100)
# p = kde.evaluate(xticks)
# ax6.plot(xticks, p, label='truth')
# samples = u2_means[1, :]
# kde = gaussian_kde(samples)
# p = kde.evaluate(xticks)
# ax6.plot(xticks, p, label='filter')

# plt.show()

/var/folders/lz/50081q0n24g8lq6ncqpbp8780000gn/T/ipykernel_19948/2050836567.py:29: ComplexWarning: Casting complex values to real discards the imaginary part
  mu_t[:, 0] = mu0
